# Stats Parallel

Notebook by Cascade Tuholske 2021.02.15 <br>
Trying to speed up 4_Event_Stats.py

In [1]:
#### Dependencies
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob
from statistics import mean
import julian
import time 
import multiprocessing as mp 
from multiprocessing import Pool
import os
import sys
import matplotlib.pyplot as plt

In [2]:
# check data
path = '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/'
fns = glob.glob(path+'*')

In [3]:
fns

['/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_1D_TREND_EXPP05.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_1D_STATS.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/WBGT32_1D_STATS.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/WBGT32_1D_TREND_ALL.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/WBGT32_1D_TREND_EXPP05.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/WBGT28_1D_TREND_EXPP05.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_1D_EXP.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/WBGT28_1D_EXP.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_2D_EXP.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_1D_TREND_ALL.json',
 '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_2D_TREND_HEATP05.json',
 '/home/cascade/projects/Urba

In [6]:
fn = fns[6]
print(fn )
for col in pd.read_json(fn, orient = 'split').columns:
    print(col)

/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_1D_EXP.json
ID_HDC_G0
year
tot_days
P
P1983
P2016
people_days
people_days_heat
people_days_pop


In [ ]:
meta_fn = '/home/cascade/projects/UrbanHeat/data/interim/GHS-UCDB-IDS.csv'

In [ ]:
meta = pd.read_csv(meta_fn)

In [ ]:
meta.head()

In [ ]:
trend_fn = '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_2D_TREND_EXPP05.json'

In [ ]:
trend = pd.read_json(trend_fn, orient = 'split')

In [ ]:
trend.head()

In [ ]:
fns

In [ ]:
df = pd.read_json(path+'HI406_2D_EXP.json', orient = 'split')
fn_nm = '/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v1/AllDATA-GHS-ERA5-HI406-PDAYS.csv'
df1 = pd.read_csv(fn_nm)

In [ ]:
city = 2046 #9691
nm = 'Paris' #Kolkata
plt.scatter(df[df['ID_HDC_G0'] == city]['year'],df[df['ID_HDC_G0'] == city]['tot_days'])
plt.title(nm+' 40.6C >=2 TOTAL DAYS/YR')
plt.xlabel('days')

In [ ]:
plt.scatter(df1[df1['ID_HDC_G0'] == city]['year'],df1[df1['ID_HDC_G0'] == city]['total_days'])
plt.title(nm+' 40.6C >=1 TOTAL DAYS/YR')
plt.xlabel('days')

In [ ]:
df1[(df1['ID_HDC_G0'] == city) & (df1['year'] == 1998)]

In [ ]:
df[(df['ID_HDC_G0'] == city) & (df['year'] == 1998)]

In [ ]:
df = pd.read_json(fns[0], orient = 'split')

In [ ]:
df.head()

In [ ]:
test = df[(df['ID_HDC_G0'] == 9691) | (df['ID_HDC_G0'] == 4417)]

In [ ]:
test.groupby(['ID_HDC_G0','year']).sum().reset_index()

In [ ]:
def tot_days(df):
    df_out = df[['ID_HDC_G0','year','duration']].groupby(['ID_HDC_G0','year']).sum().reset_index()
    
    return df_out

In [ ]:
ug = tot_days(test)
ug


In [ ]:
test = df[(df['ID_HDC_G0'] == 9691) & (df['year'] == 2016)]
test['duration'].sum()

In [ ]:
for df in dfs: print(len(df))

In [ ]:
dfs[0].head()

In [ ]:
plt.hist(dfs[0]['duration'], bins = 50);
plt.yscale('log')
plt.title('Duration all events 40.6 >=1 day')
plt.xlabel('days')

In [ ]:
tmax = dfs[0]['tmax'].to_list()
tmax_lst = [item for sublist in tmax for item in sublist]

In [ ]:
plt.hist(tmax_lst, bins = 50);
plt.yscale('log')
plt.title('HImax >=40.6C, for events >=1 day');
plt.xlabel('HI')

In [ ]:
df406 = dfs[0]
df406 = df406[df406['duration'] >= 2]

In [ ]:
plt.hist(df406['duration'], bins = 50, color = 'red');
plt.yscale('log')
plt.title('Duration all events 40.6 >=2 day')
plt.xlabel('days')

In [ ]:
tmax = df406['tmax'].to_list()
tmax_lst = [item for sublist in tmax for item in sublist]

plt.hist(tmax_lst, bins = 50);
plt.yscale('log')
plt.title('HImax >=40.6C, for events >=2 day');
plt.xlabel('HI')

In [ ]:
def add_years(df):
    """ Function adds zero to people days for all missing years for each city 
    so that regressions aren't screwed up"""
    
    years = list(np.unique(df['year'])) # Get list of all years
    row_list = []
    counter = 0
    
    for city in list(np.unique(df['ID_HDC_G0'])):
        city_id = city # Get city Id 
        city_df = df.loc[df['ID_HDC_G0'] == city] # find the location
        city_years = list(np.unique(city_df['year'])) # figure out the number of years
        
        years_dif = list(set(years) - set(city_years)) # find the missing years
        
        #print(len(years_dif))
        if len(years_dif) > 0: # add in the missing years
            
            counter = counter + len(years_dif) # counter
            
            for year in years_dif: # add rows with dummy data and zeros
                row = []
                row.append(city) # city id
                row.append(year) # missing year
                row.append(0) # total days
                row.append('np.nan') # pop year
                row.append(float(df[(df['ID_HDC_G0'] == city)]['P'+str(1983)])) # pop 83
                row.append(float(df[(df['ID_HDC_G0'] == city)]['P'+str(2016)])) # pop 16
                row.append(0) # days
                row.append(0) # pdays 83
                row.append(0) # pdays diff
                
                row_list.append(row)
    
    df_new = pd.DataFrame(row_list, columns= df.columns) # merge the new rows into a df
    
    df_new = df.append(df_new) # add the rows back to the original data frame
    
    # Updated 2020.09.07 CPT - coef can be made for heat when p in 1983 is zero
    df_new = df_new[df_new['P1983'] > 0]
    
    return df_new

In [ ]:
np.nan

In [ ]:
exp = pd.read_json(path+'HI461_1D_STATS.json', orient = 'split')

In [ ]:
exp.head()

In [ ]:
test = exp[:10000]

In [ ]:
years = list(np.unique(df['year']))
row_list = []

for city in list(np.unique(df['ID_HDC_G0'])):
    city_id = city # Get city Id 
    city_df = df.loc[df['ID_HDC_G0'] == city] # find the location
    city_years = list(np.unique(city_df['year'])) # figure out the number of years

    years_dif = list(set(years) - set(city_years)) # find the missing years
    
    if len(years_dif) > 0: # add in the missing years
        for year in years_dif: # add rows with dummy data and zeros
            row = []
            row.append(city)
            row.append(year)
            row.append(0) # duration = 0 days
            row.append(np.nan) # population for that year is not needed
            row.append(df[(df['ID_HDC_G0'] == city)]['P1983'].values[0])
            row.append(df[(df['ID_HDC_G0'] == city)]['P1983'].values[0])
            row.append(0) # people_days = 0 days
            row.append(0) # people_days_heat = 0 days
            row.append(0) # people_days_pop = 0 days
            
            row_list.append(row) # append row list
    
df_new = pd.DataFrame(row_list, columns= df.columns) # merge the new rows into a df

df_new = df.append(df_new) # add the rows back to the original data frame

# Drop any city with zero people in 1983
df_new = df_new[df_new['P1983'] > 0]

return df_new

In [ ]:
df_new

In [ ]:
set(np.unique(df_new[df_new['ID_HDC_G0'] == 12481]['year'])) - set(np.unique(test[test['ID_HDC_G0'] == 12481]['year']))

In [ ]:
set(np.unique(df_new[df_new['ID_HDC_G0'] == 12481]['year']))

In [ ]:

""" Function adds zero to people days for all missing years for each city 
so that regressions aren't screwed up"""

years = list(np.unique(exp['year'])) # Get list of all years
row_list = []
counter = 0

for city in list(np.unique(exp['ID_HDC_G0'])):
    city_id = city # Get city Id 
    city_df = exp.loc[exp['ID_HDC_G0'] == city] # find the location
    city_years = list(np.unique(city_df['year'])) # figure out the number of years

    years_dif = list(set(years) - set(city_years)) # find the missing years

    #print(len(years_dif))
    if len(years_dif) > 0: # add in the missing years

        counter = counter + len(years_dif) # counter

        for year in years_dif: # add rows with dummy data and zeros
            row = []
            row.append(city) # city id
            row.append(year) # missing year
            row.append(0) # total days
            print(exp[(exp['ID_HDC_G0'] == city)]) # pop year
#            row.append(float(exp[(exp['ID_HDC_G0'] == city)]['P'+str(year)])) # pop year
#             row.append(float(df[(df['ID_HDC_G0'] == city)]['P'+str(1983)])) # pop 83
#             row.append(float(df[(df['ID_HDC_G0'] == city)]['P'+str(2016)])) # pop 16
#             row.append(0) # days
#             row.append(0) # pdays 83
#             row.append(0) # pdays diff

#             row_list.append(row)

# df_new = pd.DataFrame(row_list, columns= df.columns) # merge the new rows into a df

# df_new = df.append(df_new) # add the rows back to the original data frame

# # Updated 2020.09.07 CPT - coef can be made for heat when p in 1983 is zero
# df_new = df_new[df_new['P1983'] > 0]



In [ ]:
out = add_years(exp)

In [ ]:
df_in = path+'HI406_2D_STATS.json'

In [ ]:
df = pd.read_json(df_in, orient = 'split')

In [ ]:
df.head()

In [ ]:
df[(df['ID_HDC_G0'] == 8716) & (df['year'] == 2000)]['duration'].sum()

In [ ]:
#### Step 1 - Function Loads all Tmax Data as an X-array
def read_data(dir_path, space_dim, time_dim):
    """ Function reads in all Tmax .csv files, joins them by date along the x-axis
    and returns the whole record as a x-array data array
    
    Args:   
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    fn_list = sorted(glob.glob(dir_path+'*.csv'))
    df_out = pd.DataFrame()
    date_list = []

    # Open all Tmax files and concat into a df
    for i, fn in enumerate(fn_list):    
        # Open the CSV
        df = pd.read_csv(fn)

        # Get the city ids 
        if i == 1:
            df_id = df[space_dim]

        # get only the Tmax columns and concate date list 
        df_temp = df.iloc[:,3:] # get only temp columns
        date_list = date_list+list(df_temp.columns)

        # Drop cities w/ no temp record 
        df_temp_drop = df_temp.dropna()

        # Merge
        df_out = pd.concat([df_out, df_temp_drop], axis=1)
        print(df_out.shape)
    
    # make date into an array
    tmax_arr = df_out.to_numpy()

    # Make data into an xr.DataArray
    tmax_xr_da = xr.DataArray(tmax_arr, coords=[df_id, date_list], 
                             dims=[space_dim, time_dim])
    return tmax_xr_da

In [ ]:
#### Step 2 Function finds all the Tmax Events and writes it to a dateframe w/ dates for each city
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # #CPT 2020.02.23 
        # dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    # df_out['total_days'] = dayTot_list #CPT 2020.02.23
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out


In [ ]:
#### Step 3 Function splits the dataset into Tmax events (continuous days >Tmax) for each city
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

def event_split(dates, ID_HDC_G0, intensity, tmax): #, total_days): #CPT 2020.02.23
    
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    # tot_days = total_days #CPT 2020.02.23
    
    # lists to fill
    city_id_list = []
    # tot_days_list = [] #CPT 2020.02.23
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    year_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = np.sum(intense) # total intensity during event 
        
        start = counter # reset start to current end (e.g. counter)
        year = event_dates[0].split('.')[0]
        
        # fill lists
        city_id_list.append(city_id)
        year_list.append(year)
        # tot_days_list.append(tot_days) #CPT 2020.02.23
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['year'] = year_list
    # df_out['total_days'] = tot_days_list #CPT 2020.02.23
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

#### Step 4 function feeds output from function 3 into function 4
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats

        NOTE - If you add arguments to event_split to make more states,
        be sure to update this function

        args:
            df: input dataframe
    """
    df_out = pd.DataFrame()

    # NOTE - If you add arguments to event_split to make more stats,
    # be sure to update this function

    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        # total_days = row['total_days'] # get total number of tmax days -- CPT 2020.02.23

        df = event_split(dates, ID_HDC_G0, intensity, tmax)# , total_days) #CPT 2020.02.23

        df_out = df_out.append(df)

    return df_out

#### Step 5 function threads it all together
def run_stats(dir_path, space_dim, time_dim, Tthresh, fn_out):
    
    """ Function ties all the Tmax Stats functions together and writes final stats for each Tmax 
    event to a .csv file. Returns results as a dataframe if needed
    
    Args:
        dir_path = path to .csv files 
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
        Tthresh = float of temp threshold
        fn_out = file and path to write final csv
        
    """
    
    # read in data
    step1= read_data(dir_path, space_dim = space_dim, time_dim = time_dim)
    #step1_sub = step1[:,:10] # subset data for testing
    print('Stack x-array made')
    
    # Mask data based on Tmax threshold ... we're using 40.6C
    step2 = tmax_days(step1, Tthresh)
    print('Tmax masked')
    
    
    # Calculate stats
    step3 = tmax_stats(step2)
    print('Stats made')

    # Save file out
    step3.to_json(fn_out, orient = 'split')
    
    return step3

    print('done')

In [ ]:
## Arges Needed 
DATA_IN = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/HI/' # output from avg temp
DATA_OUT = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/'
dir_path = DATA_IN 
space_dim = 'ID_HDC_G0'
time_dim = 'date'
Tthresh = 40.6
fn_out = DATA_OUT+'STATS_1DAY406.json'

In [ ]:
# read in data
step1 = read_data(dir_path, space_dim = space_dim, time_dim = time_dim)

In [ ]:
# Mask data based on Tmax threshold ... we're using 40.6C
step2 = tmax_days(step1, Tthresh)
print('Tmax masked')

In [ ]:
step2.head()

In [ ]:
# this is how to split the df

len(np.array_split(step2, 3)[2])

In [ ]:
fn_list = glob.glob(DATA_OUT+'HI406_tmp/*STAT*')
fn_list[0]

In [ ]:
df_list = []
for fn in fn_list:
    df_list.append(pd.read_json(fn, orient = 'split'))

df_out = pd.concat(df_list)
df_out.to_json(fn_out, orient = 'split')

In [ ]:
df_out

In [ ]:
done = pd.read_json(DATA_OUT+'HI406_STATS.json', orient = 'split')

# Is the data the same?
cpt feb 2021

In [ ]:
new = pd.read_json('/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v2/HI406_STATS.json', orient = 'split')

In [ ]:
old = pd.read_csv('/home/cascade/projects/UrbanHeat/data/processed/PNAS-DATA-v1/AllDATA-GHS-ERA5-HI406.csv')

In [ ]:
new.shape

In [ ]:
old.shape

In [ ]:
old[(old['ID_HDC_G0'] == 3091) & (old['year'] == 1983)]

In [ ]:
new[(new['ID_HDC_G0'] == 3091) & (new['year'] == 1983)]

# Old Code

In [ ]:
cpu = 20+2 # - 2
n = int(len(step2)/ cpu)  #chunk row size
list_df = [step2 [i:i+n] for i in range(0,step2.shape[0],n)]
print(len(list_df))

In [ ]:
len(step2)

In [ ]:
step2

In [ ]:
# write them out
for i, df in enumerate(list_df):
    df.to_json(DATA_OUT+'tmp/tmp_'+str(i)+'.json', orient = 'split')

In [ ]:
#os.mkdir(DATA_OUT+'temp')

In [ ]:
fns_list = glob.glob(DATA_OUT+'tmp/*.json')

In [ ]:
fns_list

In [ ]:
fn = fns_list[0]
df = pd.read_json(fn, orient = 'split')

In [ ]:
step3 = tmax_stats(df)

In [ ]:
def max_stats_run(fn):
    
    """ runs max_stats on a fn (.json) and writes on .json)
    Args:
        fn = file name
    """
    
    # open df
    df = pd.read_json(fn, orient = 'split')
    i = fn.split('temp_')[1].split('.json')[0]
    
    # make small for testing 
    df = df.iloc[0:4,:]
    
    # Calculate stats
    step3 = max_stats(df)

    # write file
    fn_out = DATA_OUT+'temp/'+'STAT_'+str(i)+'.json'
    step3.to_json(fn_out, orient = 'split')
    print('done', i)



In [ ]:
for fn in fns_list:
    tbd(fn)

In [ ]:
test = glob.glob(DATA_OUT+'temp/STAT*')

In [ ]:
test

In [ ]:
import Event_Stats_Funcs

In [ ]:
test = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_2.json'

In [ ]:
data = test.split('_temp/')[1].split('_')[0]
i = test.split(data+'_temp/')[1].split(data+'_')[1]
DATA_OUT = test.split(data+'_temp')[0]

In [ ]:
DATA_OUT

In [ ]:
glob.glob('/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/*')

In [ ]:
fn = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS_DAILY/STATS/HI406_temp/HI406_STAT_6.json'
test = pd.read_json(fn, orient = 'split')
test.head()